In [2]:
# UMLS API requires getting TGT every 8 hours: https://documentation.uts.nlm.nih.gov/rest/authentication.html
# Or just get every run (below)
import getpass
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from os import path

In [3]:
headers = {"Content-Type": "application/x-www-form-urlencoded"}

# user = input("Please enter your username: ")
# pw = getpass.getpass("Please enter your password: ")
user = 'jeffma'
pw = 'Mr13812283066'

params = {"username" : user, 
         "password" : pw}

TGT_URL = "https://utslogin.nlm.nih.gov/cas/v1/tickets"

response = (requests.post(TGT_URL, headers = headers, params = params)).text
ticketgetter = BeautifulSoup(response, 'lxml')
TGT = ticketgetter.form['action']


In [325]:
# icd-query

# ICD10: Cauda equina syndrome; Brain Injuries Traumatic
queryString = input("Please type the query (ICD10): ")


headers_ST = {"Content-Type": "application/x-www-form-urlencoded"}
params = {"service": "http://umlsks.nlm.nih.gov"}

Please type the query (ICD10): Cauda equina syndrome


In [326]:
# search the query
ST = requests.post(TGT, headers = headers_ST, params = params)

URL = "https://uts-ws.nlm.nih.gov/rest/search/current?string={}&ticket={}".format(queryString, ST.text)
response = requests.get(URL)
j = json.loads(response.text) 


# parse cui & select the first cui
mth_cui = []
for i in j['result']['results']:
    if i['rootSource'] == 'MTH':
        mth_cui.append((i['ui'],i['name']))

        
CUI = mth_cui[0][0] 
print(CUI)


ST = requests.post(TGT, headers = headers_ST, params = params)
URL_cui = "https://uts-ws.nlm.nih.gov/rest/content/current/CUI/{}/atoms?sabs=ICD10CM,MSH&ticket={}".format(CUI, ST.text)
response_2 = requests.get(URL_cui)
r2 = json.loads(response_2.text) 

C0392548


In [327]:
Mesh_terms = []
icd_terms = [] 
found_file = 'Sorry, we can not find the local files'

for atom in r2['result']: 
    if atom['rootSource'] == 'MSH':
        for i in r2['result']:
            if i['rootSource'] == 'ICD10CM':
                Mesh_terms.append(atom["name"])
                icd_terms.append(i["name"])

# Print all the matched pairs
paired_results = list(zip(Mesh_terms, icd_terms)) 
print(f'Here we found {len(paired_results)} paired results')
for i in paired_results:
    print(f'Matched: ICD10CM({i[1]}) ---> MeSH({i[0]})')

# return the files for the required Mesh terms 
for item in paired_results:
    mesh_path = ' '.join(item[0].split(',')) +'.csv'
    try:
        found_file = pd.read_csv(os.path.join('Mesh_terms',mesh_path))
        break
    except:
        print(f'MeSH term: {mesh_path} not_found')

try:
    display(found_file.head())
except:
    if paired_results:
        print(f"https://www.ncbi.nlm.nih.gov/pubmed/?term={'+'.join(paired_results[0][0].split(' '))}")
    else: 
        print(f"https://www.ncbi.nlm.nih.gov/pubmed/?term={'+'.join(queryString.split(' '))}")


Here we found 2 paired results
Matched: ICD10CM(Cauda equina syndrome) ---> MeSH(Cauda Equina Syndrome)
Matched: ICD10CM(Cauda equina syndrome) ---> MeSH(Cauda Equina Syndromes)


,Unnamed: 0,pubmed_id,authors,keywords,abstract,publication_date,title,results,doi,journal
0,0,31723404,"[('Michelle J Nentwig', 'Department of Orthopa...",[],Lumbar spine fusion has become a common and ef...,2019-11-15,Spinal subdural hygroma as a post-operative co...,NaN,10.1093/jscr/rjz305,Journal of surgical case reports
1,1,31718292,"[('Savva Pronin', 'Department of Clinical Neur...","['Cauda equina syndrome', 'disc herniation', '...",Background,2019-11-14,Does a history of lumbar spine surgery predict...,NaN,10.1080/02688697.2019.1687845,British journal of neurosurgery
2,2,31715419,"[('Asfand Baig Mirza', ""Department of Neurosur...","['CE', 'CES', 'CSF', 'Cauda Equina', 'Cauda Eq...",Cauda Equina Syndrome (CES) is a neurosurgical...,2019-11-13,Atypical cauda equina syndrome with lower limb...,NaN,10.1016/j.wneu.2019.10.198,World neurosurgery
3,3,31700690,"[('Dorothy Khai Chin Kuek', '1Derriford Hospit...","['Spinal cord', 'Stroke']",Ganglion impar block (GIB) is a well-recognise...,2019-11-09,Conus infarction after non-guided transcoccyge...,NaN,10.1038/s41394-019-0237-1,Spinal cord series and cases
4,4,31666024,"[('Lloyd Steele', 'Department of Neurosurgery,...","['Aneurysm', 'Case report', 'Dissecting', 'Fac...",Failing to recognise the signs and symptoms of...,2019-11-02,Subarachnoid haemorrhage due to intracranial v...,NaN,10.1186/s12883-019-1487-z,BMC neurology
